<a href="https://colab.research.google.com/github/JayroMartinez/game_of_life_sim/blob/main/game_of_life_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash
# CLONE OR UPDATE REPO

if [ ! -d game_of_life_sim ]; then
  git clone https://github.com/JayroMartinez/game_of_life_sim.git -q
else
  git -C game_of_life_sim pull -q
fi
pip install -q -r game_of_life_sim/requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# ENVIRONMENT SETUP
import os, sys
sys.path.append("/content/game_of_life_sim/src")
os.chdir("/content/game_of_life_sim")
print("Environment ready.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/game_of_life_sim'

In [ ]:
# SINGLE GIF SIMULATION UNTIL COLLAPSE
# --size: board side length   --prob: initial alive-cell probability
import imageio
import numpy as np
import zlib
from src.life_core import life_step
from IPython.display import Image, display

# parameters
size = 60
prob = 0.25
fps = 10

# initialize board
board = np.random.rand(size, size) < prob
frames = []
seen = {zlib.crc32(board.tobytes())}
cycles = 0

# run until collapse
while True:
    # capture frame
    frames.append((board * 255).astype('uint8'))
    # advance one step
    board = life_step(board)
    cycles += 1
    h = zlib.crc32(board.tobytes())
    # stop on extinction or cycle detection
    if not board.any() or h in seen:
        frames.append((board * 255).astype('uint8'))
        break
    seen.add(h)

print(f"Collapsed after {cycles} cycles")

# write and display GIF
gif_path = "simulation_until_collapse.gif"
imageio.mimsave(gif_path, frames, fps=fps)
display(Image(filename=gif_path))

In [ ]:
# PARALLEL SIMULATION
# --runs: Simulations to run --size: Board size length --prob: Initial alive-cell probability
!python run_sim.py --runs 100 --size 60 --prob 0.33